In [1]:
from tracemalloc import stop
import numpy as np
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier

C:\Users\LENOVO\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
nltk.download('punkt')
nltk.download('stopwords')
sw=nltk.corpus.stopwords.words("english")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
!pip install emoji


[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import emoji

In [5]:
df = pd.read_csv("UScomments.csv",error_bad_lines=False)

C:\Users\LENOVO\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3552: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 41589: expected 4 fields, saw 11\nSkipping line 51628: expected 4 fields, saw 7\nSkipping line 114465: expected 4 fields, saw 5\n'
b'Skipping line 142496: expected 4 fields, saw 8\nSkipping line 189732: expected 4 fields, saw 6\nSkipping line 245218: expected 4 fields, saw 7\n'
b'Skipping line 388430: expected 4 fields, saw 5\n'
C:\Users\LENOVO\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3552: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
df.sample(500)

video_id                                       comment_text likes  \
194910  uwukx6DGmW8       Financial advise... from unqualified people.     0   
69492   F1DxLQbx_bU                      Can you make a vegan cupcake?     0   
668505  JKZM1uTWBJw                                     Fantastic! :c)     0   
614899  lLOEu9p80rE                                  so cute i love it     0   
147250  SLoQoYi3P5U                                   TRUMP TRUMP 🇺🇸🇺🇸     3   
...             ...                                                ...   ...   
316869  zmg9tVaMVd4  Only thing swinging is her chest down to her k...     1   
390493  22tF3y_epZs       YAY this was long overdue!! Thanks for this!     0   
41560   _ANP3HR1jsM                  I think they are making a tv show     0   
609374  Xx7sxWI9FNI                            but I use a rice cooker     0   
87946   R_jKUDmJs1Y                     Dear Ryan, Can you knot a not?     0   

       replies  
194910       0  
69492        0  
668505       0  
614899       0  
147250       0  
...        ...  
316869       0  
390493       0  
41560        0  
609374       0  
87946        0  

[500 rows x 4 columns]

In [7]:
df.isnull().sum()

video_id         0
comment_text    25
likes            0
replies          0
dtype: int64

In [8]:
df.dropna(inplace=True)

In [9]:
df.duplicated().sum()

212683

In [5]:
!pip install textblob


[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from textblob import TextBlob

In [27]:
# TextBlob('i glad you').sentiment.polarity

In [28]:
# TextBlob('glad').sentiment.polarity

In [12]:
df1 = df[0:10000]

In [13]:
polarity = []
for comment in df['comment_text']:
    try:
        polarity.append(TextBlob(comment).sentiment.polarity)
    except:
        polarity.append(0)

In [14]:
polarity[0:20]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.8,
 -0.13571428571428573,
 0.0,
 0.2,
 -0.023333333333333352,
 0.5,
 0.0,
 0.8,
 -0.2916666666666667,
 0.0,
 0.25,
 -0.8]

In [15]:
df['Polarity'] = polarity

In [16]:
df.head()

video_id                                       comment_text likes  \
0  XpVt6Z1Gjjo                  Logan Paul it's yo big day ‼️‼️‼️     4   
1  XpVt6Z1Gjjo  I've been following you from the start of your...     3   
2  XpVt6Z1Gjjo                 Say hi to Kong and maverick for me     3   
3  XpVt6Z1Gjjo                                MY FAN . attendance     3   
4  XpVt6Z1Gjjo                                         trending 😉     3   

  replies  Polarity  
0       0       0.0  
1       0       0.0  
2       0       0.0  
3       0       0.0  
4       0       0.0

In [17]:
comment_positive = df[df['Polarity']==1]

In [18]:
comment_negative = df[df['Polarity']==-1]

In [19]:
df.head()

video_id                                       comment_text likes  \
0  XpVt6Z1Gjjo                  Logan Paul it's yo big day ‼️‼️‼️     4   
1  XpVt6Z1Gjjo  I've been following you from the start of your...     3   
2  XpVt6Z1Gjjo                 Say hi to Kong and maverick for me     3   
3  XpVt6Z1Gjjo                                MY FAN . attendance     3   
4  XpVt6Z1Gjjo                                         trending 😉     3   

  replies  Polarity  
0       0       0.0  
1       0       0.0  
2       0       0.0  
3       0       0.0  
4       0       0.0

In [20]:
df.Polarity.value_counts()

 0.000000    284425
 0.500000     37677
 1.000000     19390
 0.250000     12554
 0.200000     12073
              ...  
-0.072396         1
-0.050884         1
 0.087930         1
 0.065303         1
-0.115530         1
Name: Polarity, Length: 21685, dtype: int64

In [21]:
x=[]
for i in df.Polarity:
    if i>0:
        x.append('Positive')
    else:
        x.append('Negative')
#     else:
#         x.append('Neutral')
        

In [22]:
x

['Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Negative',

In [23]:
len(x)

691375

In [24]:
len(df)

691375

In [25]:
df.head()

video_id                                       comment_text likes  \
0  XpVt6Z1Gjjo                  Logan Paul it's yo big day ‼️‼️‼️     4   
1  XpVt6Z1Gjjo  I've been following you from the start of your...     3   
2  XpVt6Z1Gjjo                 Say hi to Kong and maverick for me     3   
3  XpVt6Z1Gjjo                                MY FAN . attendance     3   
4  XpVt6Z1Gjjo                                         trending 😉     3   

  replies  Polarity  
0       0       0.0  
1       0       0.0  
2       0       0.0  
3       0       0.0  
4       0       0.0

In [26]:
df['Target'] = x

In [27]:
df.head()

video_id                                       comment_text likes  \
0  XpVt6Z1Gjjo                  Logan Paul it's yo big day ‼️‼️‼️     4   
1  XpVt6Z1Gjjo  I've been following you from the start of your...     3   
2  XpVt6Z1Gjjo                 Say hi to Kong and maverick for me     3   
3  XpVt6Z1Gjjo                                MY FAN . attendance     3   
4  XpVt6Z1Gjjo                                         trending 😉     3   

  replies  Polarity    Target  
0       0       0.0  Negative  
1       0       0.0  Negative  
2       0       0.0  Negative  
3       0       0.0  Negative  
4       0       0.0  Negative

In [28]:
df.iloc[498]['comment_text']

'I think the room would have been a better movie if Johnny was a vampire'

In [29]:
df.iloc[498]['Target']

'Positive'

In [30]:
import string

In [228]:
# import pickle
# with open('Emoji_Dict.p', 'rb') as fp:
#     Emoji_Dict = pickle.load(fp)
# Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

# def convert_emojis_to_word(text):
#     for emot in Emoji_Dict:
#         text = re.sub(r'('+emot+')', "_".join(Emoji_Dict[emot].replace(",","").replace(":","").split()), text)
#     return text

# text = "I won 🥇 in 🏏"
# convert_emojis_to_word(text)

'I won 1st_place_medal in cricket'

In [31]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score

In [32]:
import re

In [254]:
def devide_emoji_with_text(str):
    new_list_emoji_df = [i for i in new_df['Char']]
#     print(new_list_emoji_df)
    text_list_extract=[]
    emoji_list_extract=[]
#     c=0
    for i in str:
#         print(c)
        if i in new_list_emoji_df:
            emoji_list_extract.append(i)
        else:
            text_list_extract.append(i)
#         c+=1
            
    text = "".join(text_list_extract)
#     print(c)
    return len(emoji_list_extract),emoji_list_extract,text




no,emoji_list,text=devide_emoji_with_text('The end though happy😭👍')

print(no)
print(emoji_list)
print(text)


2
['😭', '👍']
The end though happy


In [255]:
def transform_text(text):
#     text1 = convert_emojis_to_word(text)
#     text.encode('utf-8')
    text1=text.lower()
    text1=nltk.word_tokenize(text1)
#     print(text1)
    y=[]
    for i in text1:
#         if i.isalnum():
#         if i.islower() or i.encode('utf-8'):
        if i.islower():
            y.append(i)
    text1=y[:]
#     print(text1)
    y.clear()
    for i in text1:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)
#         if i in emoji.distinct_emoji_list(i):
#             y.append(i)
    text1=y[:]
#     print(text1)
    y.clear()
    ps=PorterStemmer()
    for i in text1:
        y.append(ps.stem(i))
#     print(text1)
    return " ".join(y)

In [256]:
transform_text(text)

'end though happi'

In [175]:
# if emoji.get_emoji_regexp:
#     print("yes")

In [274]:
tfidf1=TfidfVectorizer(stop_words=sw,max_features=20)
def transform1(txt1):
    txt2=tfidf1.fit_transform(txt1)
    return txt2.toarray()


In [73]:
x=transform1(df["comment_text"])
y=df["Target"]
x_train1,x_test1,y_train1,y_test1=train_test_split(x,y,test_size=0.2,random_state=0)

In [258]:
model1=MultinomialNB()
model1.fit(x_train1,y_train1)

MultinomialNB()

In [43]:
# emoji_df = pd.read_csv("__readervswriter.csv")

In [44]:
# emoji_df.head()

In [45]:
# emoji_df.isnull().sum()

In [46]:
# emoji_df.drop(['Unnamed: 7'],axis=1,inplace=True)

In [47]:
# emoji_df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [48]:
# emoji_df.head()

In [49]:
# emoji_df.emoji.value_counts()

In [347]:
!pip install emoji


[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [55]:
new_emoji_df = pd.read_csv(r"ijstable.csv")

In [56]:
new_df = new_emoji_df[1:]

In [57]:
new_df.reset_index().drop(['index'],axis=1)

Char  Unnamed: 1  Unicode Occurrences Position    Neg   Neut    Pos  \
0      😂         NaN  0x1f602       14622    0.805  0.247  0.285  0.468   
1      ❤         NaN   0x2764        8050    0.747  0.044  0.166   0.79   
2      ♥         NaN   0x2665        7144    0.754  0.035  0.272  0.693   
3      😍         NaN  0x1f60d        6359    0.765  0.052  0.219  0.729   
4      😭         NaN  0x1f62d        5526    0.803  0.436   0.22  0.343   
..   ...         ...      ...         ...      ...    ...    ...    ...   
746    ♮         NaN   0x266e           5    0.937  0.125  0.625   0.25   
747    🅾         NaN  0x1f17e           5    0.977  0.375  0.375   0.25   
748    🔄         NaN  0x1f504           5    0.971  0.125   0.75  0.125   
749    ☄         NaN   0x2604           5    0.435  0.125   0.75  0.125   
750    ☨         NaN   0x2628           5    0.458  0.125   0.75  0.125   

    Sentiment score Sentiment bar  \
0             0.221           NaN   
1             0.746           NaN   
2             0.657           NaN   
3             0.678           NaN   
4            -0.093           NaN   
..              ...           ...   
746           0.125           NaN   
747          -0.125           NaN   
748               0           NaN   
749               0           NaN   
750               0           NaN   

                                          Unicode name  \
0                               FACE WITH TEARS OF JOY   
1                                    HEAVY BLACK HEART   
2                                     BLACK HEART SUIT   
3                  SMILING FACE WITH HEART-SHAPED EYES   
4                                   LOUDLY CRYING FACE   
..                                                 ...   
746                                 MUSIC NATURAL SIGN   
747            NEGATIVE SQUARED LATIN CAPITAL LETTER O   
748  ANTICLOCKWISE DOWNWARDS AND UPWARDS OPEN CIRCL...   
749                                              COMET   
750                                  CROSS OF LORRAINE   

                             Unicode block  
0                                Emoticons  
1                                 Dingbats  
2                    Miscellaneous Symbols  
3                                Emoticons  
4                                Emoticons  
..                                     ...  
746                  Miscellaneous Symbols  
747       Enclosed Alphanumeric Supplement  
748  Miscellaneous Symbols and Pictographs  
749                  Miscellaneous Symbols  
750                  Miscellaneous Symbols  

[751 rows x 12 columns]

In [277]:
new_df.head(20)

Char  Unnamed: 1  Unicode Occurrences Position    Neg   Neut    Pos  \
1     😂         NaN  0x1f602       14622    0.805  0.247  0.285  0.468   
2     ❤         NaN   0x2764        8050    0.747  0.044  0.166   0.79   
3     ♥         NaN   0x2665        7144    0.754  0.035  0.272  0.693   
4     😍         NaN  0x1f60d        6359    0.765  0.052  0.219  0.729   
5     😭         NaN  0x1f62d        5526    0.803  0.436   0.22  0.343   
6     😘         NaN  0x1f618        3648    0.854  0.053  0.193  0.754   
7     😊         NaN  0x1f60a        3186    0.813   0.06  0.237  0.704   
8     👌         NaN  0x1f44c        2925    0.805  0.094  0.249  0.657   
9     💕         NaN  0x1f495        2400    0.766  0.042  0.285  0.674   
10    👏         NaN  0x1f44f        2336    0.787  0.104  0.271  0.624   
11    😁         NaN  0x1f601        2189    0.796  0.127  0.296  0.577   
12    ☺         NaN   0x263a        2062    0.799  0.062  0.218   0.72   
13    ♡         NaN   0x2661        1975    0.764  0.052  0.227  0.721   
14    👍         NaN  0x1f44d        1854    0.812  0.115  0.248  0.637   
15    😩         NaN  0x1f629        1808    0.826  0.591  0.186  0.223   
16    🙏         NaN  0x1f64f        1539    0.794  0.081  0.421  0.498   
17    ✌         NaN   0x270c        1534     0.79  0.113   0.31  0.576   
18    😏         NaN  0x1f60f        1522    0.765  0.112  0.444  0.444   
19    😉         NaN  0x1f609        1521    0.845    0.1  0.337  0.563   
20    🙌         NaN  0x1f64c        1506    0.791  0.101  0.238  0.661   

   Sentiment score Sentiment bar                              Unicode name  \
1            0.221           NaN                    FACE WITH TEARS OF JOY   
2            0.746           NaN                         HEAVY BLACK HEART   
3            0.657           NaN                          BLACK HEART SUIT   
4            0.678           NaN       SMILING FACE WITH HEART-SHAPED EYES   
5           -0.093           NaN                        LOUDLY CRYING FACE   
6            0.701           NaN                      FACE THROWING A KISS   
7            0.644           NaN            SMILING FACE WITH SMILING EYES   
8            0.563           NaN                              OK HAND SIGN   
9            0.632           NaN                                TWO HEARTS   
10            0.52           NaN                       CLAPPING HANDS SIGN   
11           0.449           NaN           GRINNING FACE WITH SMILING EYES   
12           0.657           NaN                        WHITE SMILING FACE   
13           0.669           NaN                          WHITE HEART SUIT   
14           0.521           NaN                            THUMBS UP SIGN   
15          -0.368           NaN                                WEARY FACE   
16           0.417           NaN                  PERSON WITH FOLDED HANDS   
17           0.463           NaN                              VICTORY HAND   
18           0.332           NaN                             SMIRKING FACE   
19           0.463           NaN                              WINKING FACE   
20           0.559           NaN  PERSON RAISING BOTH HANDS IN CELEBRATION   

                            Unicode block  
1                               Emoticons  
2                                Dingbats  
3                   Miscellaneous Symbols  
4                               Emoticons  
5                               Emoticons  
6                               Emoticons  
7                               Emoticons  
8   Miscellaneous Symbols and Pictographs  
9   Miscellaneous Symbols and Pictographs  
10  Miscellaneous Symbols and Pictographs  
11                              Emoticons  
12                  Miscellaneous Symbols  
13                  Miscellaneous Symbols  
14  Miscellaneous Symbols and Pictographs  
15                              Emoticons  
16                              Emoticons  
17                               Dingbats  
18               

In [76]:
new_list_emoji_df = [i for i in new_df['Char']]

In [77]:
new_list_emoji_df

['😂',
 '❤',
 '♥',
 '😍',
 '😭',
 '😘',
 '😊',
 '👌',
 '💕',
 '👏',
 '😁',
 '☺',
 '♡',
 '👍',
 '😩',
 '🙏',
 '✌',
 '😏',
 '😉',
 '🙌',
 '🙈',
 '💪',
 '😄',
 '😒',
 '💃',
 '💖',
 '😃',
 '😔',
 '😱',
 '🎉',
 '😜',
 '☯',
 '🌸',
 '💜',
 '💙',
 '✨',
 '😳',
 '💗',
 '★',
 '█',
 '☀',
 '😡',
 '😎',
 '😢',
 '💋',
 '😋',
 '🙊',
 '😴',
 '🎶',
 '💞',
 '😌',
 '🔥',
 '💯',
 '🔫',
 '💛',
 '💁',
 '💚',
 '♫',
 '😞',
 '😆',
 '😝',
 '😪',
 '�',
 '😫',
 '😅',
 '👊',
 '💀',
 '😀',
 '😚',
 '😻',
 '©',
 '👀',
 '💘',
 '🐓',
 '☕',
 '👋',
 '✋',
 '🎊',
 '🍕',
 '❄',
 '😥',
 '😕',
 '💥',
 '💔',
 '😤',
 '😈',
 '►',
 '✈',
 '🔝',
 '😰',
 '⚽',
 '😑',
 '👑',
 '😹',
 '👉',
 '🍃',
 '🎁',
 '😠',
 '🐧',
 '☆',
 '🍀',
 '🎈',
 '🎅',
 '😓',
 '😣',
 '😐',
 '✊',
 '😨',
 '😖',
 '💤',
 '💓',
 '👎',
 '💦',
 '✔',
 '😷',
 '⚡',
 '🙋',
 '🎄',
 '💩',
 '🎵',
 '➡',
 '😛',
 '😬',
 '👯',
 '💎',
 '🌿',
 '🎂',
 '🌟',
 '🔮',
 '❗',
 '👫',
 '🏆',
 '✖',
 '☝',
 '😙',
 '⛄',
 '👅',
 '♪',
 '🍂',
 '💏',
 '🔪',
 '🌴',
 '👈',
 '🌹',
 '🙆',
 '➜',
 '👻',
 '💰',
 '🍻',
 '🙅',
 '🌞',
 '🍁',
 '⭐',
 '▪',
 '🎀',
 '━',
 '☷',
 '🐷',
 '🙉',
 '🌺',
 '💅',
 '🐶',
 '🌚',
 '👽',
 '🎤',
 '👭',
 '🎧'

In [191]:
# def devide_emoji_with_text(str):
#     new_list_emoji_df = [i for i in new_df['Char']]
    
#     text_list_extract=[]
#     emoji_list_extract=[]
#     for i in str:
#         if i in new_list_emoji_df:
#             emoji_list_extract.append(i)
#         else:
#             text_list_extract.append(i)
            
#     text = "".join(text_list_extract)
#     return len(emoji_list_extract),emoji_list_extract,text


In [261]:
no,list_emoji,txt = devide_emoji_with_text('The hello i am happy😭👍')

#no,list_emoji,txt = devide_emoji_with_text('The hello')

In [262]:
print(no)
print(list_emoji)
print(txt)

2
['😭', '👍']
The hello i am happy


In [265]:
new_str = transform_text(txt)

In [266]:
new_str

'hello happi'

In [267]:
TextBlob(new_str).sentiment.polarity

0.0

In [276]:
# tmp = tfidf1.transform([new_str])
tmp = tfidf1.transform([new_str])

NotFittedError: The TF-IDF vectorizer is not fitted

In [199]:
pred = model1.predict(tmp.todense())

In [200]:
pred[0]

'Negative'

In [186]:
def fun1(str):
#     example_str = str
    new_str = transform_text(str)
    tmp = tfidf1.transform([new_str])
    pred = model1.predict(tmp.todense())
    
    if pred[0]=="Positive":
        return 1
    else:
        return -1

In [187]:
fun1(txt)

NotFittedError: The TF-IDF vectorizer is not fitted

In [204]:
def fun2(num,list_of_extract_emoji):
#     no,emojis = extract_emojis(str)
#     list_emoji_df = [i for i in emoji_df['Emoji_Symbole']]
    sent=0
    cnt=0
    if num==0:
        return 0
    for i in list_of_extract_emoji:
        if i in new_list_emoji_df:
            if new_df.iloc[cnt]['Pos']>new_df.iloc[cnt]['Neg']:
                sent+=float(new_df.iloc[cnt]['Pos'])
    #         print(sent)
    #         print(i)
            else:
                sent+=float(new_df.iloc[cnt]['Neg'])
        else:
            sent+=0
    #         print(0)
        cnt+=1
    sent /= num
    
    if sent>0:
        return 1
    else:
        return -1

 

In [205]:
fun2(no,list_emoji)

1

In [116]:
def final_result(str):
    
    def devide_emoji_with_text(str):
        new_list_emoji_df = [i for i in new_df['Char']]

        text_list_extract=[]
        emoji_list_extract=[]
        for i in str:
            if i in new_list_emoji_df:
                emoji_list_extract.append(i)
            else:
                text_list_extract.append(i)

        text = "".join(text_list_extract)
        return len(emoji_list_extract),emoji_list_extract,text

    no,list_emoji,text = devide_emoji_with_text(str)
    
    
    
    if (fun1(text)+fun2(no,list_emoji))/2>0:
        return "Positive"
    elif (fun1(text)+fun2(no,list_emoji))/2<0:
        return "Negative"
    else:
        return "Netural"

In [117]:
final_result('')

'Negative'

In [240]:
df.head()

video_id                                       comment_text likes  \
0  XpVt6Z1Gjjo                  Logan Paul it's yo big day ‼️‼️‼️     4   
1  XpVt6Z1Gjjo  I've been following you from the start of your...     3   
2  XpVt6Z1Gjjo                 Say hi to Kong and maverick for me     3   
3  XpVt6Z1Gjjo                                MY FAN . attendance     3   
4  XpVt6Z1Gjjo                                         trending 😉     3   

  replies  Polarity    Target  
0       0       0.0  Negative  
1       0       0.0  Negative  
2       0       0.0  Negative  
3       0       0.0  Negative  
4       0       0.0  Negative

In [248]:
# len(df[df['Polarity']==0.0])/len(df)

In [252]:
len(df[df['Target']=='Positive'])/len(df)

0.4350692460676189

In [253]:
len(df[df['Target']=='Negative'])/len(df)

0.5649307539323811

In [247]:
df.Target.value_counts()

Negative    390579
Positive    300796
Name: Target, dtype: int64